**PREPARE ENVIRONMENT**

- You can set **parameters_in_vram** to 10 or less to reduce the VRAM used, especially if you want to generate a video with a resolution greater than 480 by 832 or 832 by 480. The VRAM used depends on the value of **parameters_in_vram** and the resolutions of the input image & output video.
- Setting **parameters_in_vram** to 15 with a video resolution of 480 by 832 will result in a generation time of approximately 10 minutes. Lower values of **parameters_in_vram** will result in longer generation times. Higher values can reduce the video generation time, but increase the risk of getting **Out of Memory Error**s.

In [ ]:
# @title
!git clone https://github.com/Isi-dev/DiffSynth-Studio.git
%cd DiffSynth-Studio
!pip install -e .
!pip install "huggingface_hub[cli]"
!apt-get install -y aria2
import os
from huggingface_hub import list_repo_files

repo_id = "Isi99999/Wan2.1-I2V-14B-480P"
all_files = list_repo_files(repo_id)
base_url = f"https://huggingface.co/{repo_id}/resolve/main/"

with open("file_list.txt", "w") as f:
    for file_path in all_files:
        full_url = f"{base_url}{file_path}"
        save_path = f"models/Wan-AI/Wan2.1-I2V-14B-480P/{file_path}"
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        f.write(f"{full_url}\n out={save_path}\n")
!aria2c -x 16 -s 16 -i file_list.txt --continue=true --auto-file-renaming=false

print("✅ All models downloaded successfully!")

import torch
from diffsynth import ModelManager, WanVideoPipeline, VideoData, save_video

model_manager = ModelManager(device="cpu")
model_manager.load_models(
    ["models/Wan-AI/Wan2.1-I2V-14B-480P/models_clip_open-clip-xlm-roberta-large-vit-huge-14.pth"],
    torch_dtype=torch.float16, # Image Encoder is loaded with float16
)
model_manager.load_models(
    [
        "models/Wan-AI/Wan2.1-I2V-14B-480P/diffusion_pytorch_model.safetensors",
        "models/Wan-AI/Wan2.1-I2V-14B-480P/models_t5_umt5-xxl-enc-bf16.safetensors",
        "models/Wan-AI/Wan2.1-I2V-14B-480P/Wan2.1_VAE.safetensors",
    ],
    torch_dtype=torch.torch.bfloat16, # You can set `torch_dtype=torch.float8_e4m3fn` or `torch_dtype=torch.bfloat16` to disable FP8 quantization.
)

pipe = WanVideoPipeline.from_model_manager(model_manager, torch_dtype=torch.bfloat16, device="cuda")
parameters_in_vram = 14 # @param {"type":"number"}
pipe.enable_vram_management(num_persistent_param_in_dit=parameters_in_vram*10**9) # You can set `num_persistent_param_in_dit` to a small number to reduce VRAM required.
print("✅ All models loaded successfully!")

**RUN TO UPLOAD IMAGE**

In [ ]:
# @title
from google.colab import files
from PIL import Image

uploaded = files.upload()
image_path = list(uploaded.keys())[0]
image = Image.open(image_path)

print("✅Image loaded successfully:", image.size)



**RUN IMAGE TO VIDEO**

In [ ]:


prompt = "The lady smiles and waves at the camera." # @param {type:"string"}
sample_steps = 30 # @param {"type":"number"}
Instruction = "choose from '720*1280', '1280*720', '480*832', '832*480', '1024*1024 for output video's width & height." # @param {"type":"string"}
width = 480 # @param {"type":"number"}
height = 832 # @param {"type":"number"}
num_frames = 81 # @param {"type":"number"}
seed = 1 # @param {"type":"number"}

# Generate video from text prompt and Image
video = pipe(
    prompt=prompt,
    negative_prompt="色调艳丽，过曝，静态，细节模糊不清，字幕，风格，作品，画作，画面，静止，整体发灰，最差质量，低质量，JPEG压缩残留，丑陋的，残缺的，多余的手指，画得不好的手部，画得不好的脸部，畸形的，毁容的，形态畸形的肢体，手指融合，静止不动的画面，杂乱的背景，三条腿，背景人很多，倒着走",
    input_image=image,
    height = height,
    width = width,
    num_frames=num_frames,
    num_inference_steps=sample_steps,
    seed=seed, tiled=True
)

# # Save the generated video
save_video(video, "video.mp4", fps=15, quality=5)

from IPython.display import display as displayVid, Video as outVid
import os

# Function to display video
def show_video(video_path):
    if os.path.exists(video_path):
        displayVid(outVid(video_path, embed=True))
    else:
        print(f"Error: {video_path} not found!")

# Show the video
show_video("video.mp4")